# Creating spreadsheet from B3 to income tax

# Criando uma planilha com dados retirados da B3 para declarar no impost do renda

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

Is necessary to put the data in raw_data directory because the informations are personas

Necessário colocar os dados em raw_data por se tratar de informações pessoais

In [2]:
path = '../raw_data'
files = [f for f in listdir(path) if isfile(join(path, f))] 

Feature engineering

Organizando as colunas 

In [3]:
list_files = []
for file in range(0, len(files)):
    list_file = pd.read_excel(f'{path}/{files[file]}')
    list_files.append(list_file)
list_all_files = pd.concat(list_files)
list_all_files.Data = pd.to_datetime(list_all_files.Data)
list_files_s = list_all_files.sort_values(by=['Produto', 'Data'])
df_files = pd.DataFrame(list_files_s)
df_files

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
48,Debito,2020-05-19,Transferência - Liquidação,ABEV3 - AMBEV S.A. ...,XP INVESTIMENTOS CCTVM S/A,100,11.81,1181.00
6,Debito,2021-10-11,Transferência - Liquidação,AMER3 - B2W COMPANHIA DIGITAL,CLEAR CORRETORA - GRUPO XP,72,32.67,2352.24
31,Credito,2020-03-06,Transferência - Liquidação,BBAS3 - BCO BRASIL S.A. ...,CLEAR CORRETORA - GRUPO XP,50,31.86,1593.00
17,Credito,2020-06-24,Transferência - Liquidação,BBAS3 - BCO BRASIL S.A. ...,CLEAR CORRETORA - GRUPO XP,50,33.70,1685.00
7,Debito,2021-05-24,Transferência - Liquidação,BBAS3 - BCO BRASIL S.A. ...,CLEAR CORRETORA - GRUPO XP,210,32.83,6894.30
...,...,...,...,...,...,...,...,...
62,Credito,2020-03-20,Transferência - Liquidação,VVAR3 - VIA VAREJO S.A. ...,CLEAR CORRETORA - GRUPO XP,900,4.35,3915.00
50,Debito,2020-05-05,Transferência - Liquidação,VVAR3 - VIA VAREJO S.A. ...,CLEAR CORRETORA - GRUPO XP,1800,9.00,16200.00
41,Credito,2020-03-06,Transferência - Liquidação,YDUQ3 - YDUQS PARTICIPACOES S.A. ...,CLEAR CORRETORA - GRUPO XP,40,29.50,1180.00
14,Credito,2020-07-28,Transferência - Liquidação,YDUQ3 - YDUQS PARTICIPACOES S.A. ...,CLEAR CORRETORA - GRUPO XP,40,32.98,1319.20


In [4]:
df_files['Data da compra'] = df_files['Data'].where(df_files['Entrada/Saída'] == 'Credito')
df_files['Data da venda'] = df_files['Data'].where(df_files['Entrada/Saída'] == 'Debito')
df_files = df_files[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Movimentação', 'Instituição']]
df_files['Quantidade'] = df_files['Quantidade'].apply(lambda x: x.replace(',','.'))
df_files['Quantidade'] = df_files['Quantidade'].astype(float)
df_stock = df_files[df_files['Movimentação'] == 'Transferência - Liquidação']
df_stock = df_files[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Instituição']].reset_index()
df_stock

/tmp/ipykernel_750/4106693912.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_files['Quantidade'] = df_files['Quantidade'].apply(lambda x: x.replace(',','.'))
/tmp/ipykernel_750/4106693912.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_files['Quantidade'] = df_files['Quantidade'].astype(float)


In [5]:
df_fixedi_c = df_files[df_files['Movimentação'] == 'Compra']
df_fixedi_v = df_files[df_files['Movimentação'] == 'Venda']
df_fixedi = pd.concat([df_fixedi_c, df_fixedi_v])
df_fixedi = df_fixedi[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Instituição']]
df_fixedi = df_fixedi.sort_values(by=['Produto']).reset_index()
df_fixedi

,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Instituição
0,63,Tesouro Selic 2023,NaT,2020-03-17,0.71,10551.64,7491.66,XP INVESTIMENTOS CCTVM S/A
1,78,Tesouro Selic 2025,2020-07-01,NaT,2.14,10466.52,22398.35,CLEAR CORRETORA - GRUPO XP
2,0,Tesouro Selic 2025,2022-03-23,NaT,4.72,11457.89,54081.24,CLEAR CORRETORA - GRUPO XP
3,77,Tesouro Selic 2025,NaT,2020-03-02,2.14,10495.41,22460.18,CLEAR CORRETORA - GRUPO XP
4,6,Tesouro Selic 2027,2021-10-19,NaT,7.34,10905.72,80047.98,CLEAR CORRETORA - GRUPO XP


In [43]:
Saldo_de_acoes = df_fixedi.groupby('Produto').agg({'Quantidade':'sum'})
Saldo_de_acoes

,Quantidade
Produto,
Tesouro Selic 2023,0.71
Tesouro Selic 2025,9.00
Tesouro Selic 2027,7.34


In [ ]:
# create a lucro feature

In [10]:
# Not work weel however was amazing
# df_fixedi['Saldo_de_acoes'] = ''
# for line in range(0, len(df_fixedi)):
#     if df_fixedi['Produto'].iloc[line] != df_fixedi['Produto'].iloc[line+1]:
#         df_fixedi['Saldo_de_acoes'].iloc[line] = Saldo_de_acoes['Quantidade'].iloc[0]
#     else:
#         df_fixedi['Saldo_de_acoes'].iloc[line] = 'nan'
#     if df_fixedi['Produto'].iloc[-1] == df_fixedi['Produto'].iloc[-2]:
#         df_fixedi['Saldo_de_acoes'].iloc[-1] = Saldo_de_acoes['Quantidade'].iloc[-1]

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


IndexError: single positional indexer is out-of-bounds

In [ ]:
df_fixedi.Quantidade[df_fixedi.Produto == df_fixedi.Produto].sum()

In [42]:
df_fixedi

,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Instituição,Saldo_de_acoes
0,63,Tesouro Selic 2023,NaT,2020-03-17,0.71,10551.64,7491.66,XP INVESTIMENTOS CCTVM S/A,0.71
1,78,Tesouro Selic 2025,2020-07-01,NaT,2.14,10466.52,22398.35,CLEAR CORRETORA - GRUPO XP,nan
2,0,Tesouro Selic 2025,2022-03-23,NaT,4.72,11457.89,54081.24,CLEAR CORRETORA - GRUPO XP,nan
3,77,Tesouro Selic 2025,NaT,2020-03-02,2.14,10495.41,22460.18,CLEAR CORRETORA - GRUPO XP,0.71
4,6,Tesouro Selic 2027,2021-10-19,NaT,7.34,10905.72,80047.98,CLEAR CORRETORA - GRUPO XP,


In [ ]:
#creating spreadsheet
#nome cnpj data da compra data da venda quantidade valor R$ total lucro ou prejuízo R$ saldo de de ações
b3_sheet = 